In [4]:
import pandas as pd 
import numpy 

In [11]:
file_path = 'keylog.txt'
columns = ['Timestamp', 'Key', 'Action']
dataframe = pd.read_csv(file_path, sep=";")

display(dataframe)

,"2021-10-19 11:27:20,755",'a',press
0,"2021-10-19 11:27:20,875",'a',release
1,"2021-10-19 11:27:21,024",'a',press
2,"2021-10-19 11:27:21,143",'a',release
3,"2021-10-19 11:27:21,285",'a',press
4,"2021-10-19 11:27:21,405",'a',release
...,...,...,...
108,"2021-10-19 11:30:58,384",Key.enter,release
109,"2021-10-19 11:31:11,656",Key.ctrl_l,press
110,"2021-10-19 11:31:11,904",'\x17',press
111,"2021-10-19 11:31:12,022",'\x17',release
